# 参数管理

In [23]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.1366],
        [0.1305]], grad_fn=<AddmmBackward0>)

In [24]:
""" 访问参数 """
# Sequential 类似 List
print(net[2].state_dict())
print(net[2].weight.shape)
print(net[2].state_dict()['bias'].shape)

OrderedDict([('weight', tensor([[ 0.0834, -0.0422,  0.2536, -0.1464, -0.1295,  0.2063,  0.0497, -0.2062]])), ('bias', tensor([0.0625]))])
torch.Size([1, 8])
torch.Size([1])


In [25]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data, '\t', net[2].bias.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0625], requires_grad=True)
tensor([0.0625]) 	 None


In [26]:
""" 一次性访问所有的参数 """
print(*[(name, param.shape) for name, param in net.named_parameters()]) # ReLU 没有参数

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


嵌套块

In [27]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1()) # add_module 可以为每一层指定标记
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(torch.rand(size=(2, 4)))

tensor([[0.3792],
        [0.3793]], grad_fn=<AddmmBackward0>)

In [28]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 初始化

`normal_`的`_`表示原地操作，而不是返回操作值

In [29]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal) # 一层层的应用函数
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0235,  0.0025, -0.0032, -0.0056]), tensor(0.))

In [30]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1) # w 完全相同，相当于是单层网络
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

### 不同层不同初始化

In [33]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight) # 使用均匀分布的 Xavier

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)


net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-0.2320, -0.3438, -0.0325, -0.2570],
        [ 0.4182,  0.0540, -0.5864,  0.6722],
        [ 0.2393,  0.2716,  0.5169,  0.1446],
        [ 0.5215, -0.2278, -0.5817, -0.6974],
        [-0.0645, -0.6964, -0.1587,  0.3428],
        [ 0.6648, -0.0874,  0.4238,  0.0494],
        [ 0.2888,  0.6725,  0.3133,  0.1552],
        [-0.0539, -0.5681, -0.6611, -0.5765]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [34]:
""" 自定义初始化 """
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5 # 保留绝对值大于5的值

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000, -0.0000, -9.5053],
        [ 0.0000, -8.2868,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [37]:
""" 参数绑定(利用浅拷贝共享权重) """
shared = nn.Linear(8, 8)

net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    shared, 
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8, 1)
)

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
